# A. Build First Model (no data preparation)

### Install packages / import namespaces

In [1]:
#r "nuget:Microsoft.ML"
using Microsoft.ML;
using Microsoft.ML.Data;

Installed package Microsoft.ML version 1.4.0

### Load data models from Models.cs file 

In [2]:
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Models.cs"

### Initialize the ML context (we need it for building the data and training pipelines)  

In [3]:
MLContext mlContext = new MLContext(0);

### Load data from csv file into a dataview

In [4]:
const string DATASET_PATH = "./sensors_data.csv";
IDataView data = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: DATASET_PATH,
    hasHeader: true,
    separatorChar: ',');

### Shuffle and split data to _trainingData_ and _testingData_ by a fraction of 0.2

In [5]:
var shuffledData = mlContext.Data.ShuffleRows(data, seed: 0);
var split = mlContext.Data.TrainTestSplit(shuffledData, testFraction: 0.2);
var trainingData = split.TrainSet;
var testingData = split.TestSet;

### Model builder pipeline

In [6]:
var featureColumns = new string[] { "Temperature", "Luminosity", "Infrared" };

var pipeline = mlContext.Transforms.Conversion.MapValueToKey("Label")
    .Append(mlContext.Transforms.Concatenate("Features", featureColumns))
    .Append(mlContext.Transforms.NormalizeMinMax("Features"))
    .Append(mlContext.MulticlassClassification.Trainers.SdcaNonCalibrated("Label", "Features"))
    .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

### Train the model

In [7]:
var model = pipeline.Fit(trainingData);

In [8]:
var predictions = model.Transform(testingData);
var metrics = mlContext.MulticlassClassification.Evaluate(predictions, "Label", "Score", "PredictedLabel");

In [9]:
#r "nuget:Microsoft.Data.Analysis"
using Microsoft.AspNetCore.Html;
using Microsoft.Data.Analysis;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using System.Collections.Generic;
using static Microsoft.ML.TrainCatalogBase;

Installed package Microsoft.Data.Analysis version 0.4.0

In [10]:
Formatter<MulticlassClassificationMetrics>.Register((m, writer) =>
{
    var oneMessage = "the closer to 1, the better";
    var zeroMessage = "the closer to 0, the better";
    var categories = new string[] { "FlashLight", "Infrared", "Day", "Lighter" };

    var headers = new List<IHtmlContent>
    {
        th("EVALUATION: Metrics for multi-class classification model")
    };

    var rows = new List<List<IHtmlContent>>();

    var cells = new List<IHtmlContent>
    {
        td(b("Metric")),
        td(b("Class")),
        td(b("Value")),
        td(b("Note"))
    };
    rows.Add(cells);

    cells = new List<IHtmlContent>
    {
        td(b("MacroAccuracy")),
        td(""),
        td($"{m.MacroAccuracy:0.000}"),
        td(oneMessage)
    };
    rows.Add(cells);

    cells = new List<IHtmlContent>
    {
        td(b("MicroAccuracy")),
        td(""),
        td($"{m.MicroAccuracy:0.000}"),
        td(oneMessage)
    };
    rows.Add(cells);

    cells = new List<IHtmlContent>
    {
        td(b("LogLoss")),
        td(""),
        td($"{m.LogLoss:0.000}"),
        td(zeroMessage)
    };
    rows.Add(cells);

    cells = new List<IHtmlContent>
    {
        td[rowspan: $"{m.PerClassLogLoss.Count + 1}"](b("LogLoss by Class"))
    };
    rows.Add(cells);

    for (int i = 0; i < m.PerClassLogLoss.Count; i++)
    {
        cells = new List<IHtmlContent>
        {
            td($"{categories[i]}"),
            td($"{m.PerClassLogLoss[i]:0.000}"),
            td(zeroMessage)
        };
        rows.Add(cells);
    }

    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    writer.Write(t);
}, "text/html");

metrics

EVALUATION: Metrics for multi-class classification model Metric Class Value Note MacroAccuracy 0.968 the closer to 1, the better MicroAccuracy 0.960 the closer to 1, the better LogLoss 0.034 the closer to 0, the better LogLoss by Class FlashLight 0.071 the closer to 0, the better Infrared 0.000 the closer to 0, the better Day 0.000 the closer to 0, the better Lighter 0.043 the closer to 0, the better

In [11]:
var sampleData = new ModelInput
{
    Luminosity = 92F,
    Temperature = 122F,
    Infrared = 0F
};

var predictor = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(model);
var predicted = predictor.Predict(sampleData);

display(p[style: "font-size: 24px"]($"{predicted.PredictedLabel}"));

FlashLight